In [ ]:
!pip install -r requirements.txt
!pip install IPython

In [1]:
from utils import get_openai_api_key, get_anthropic_api_key
OPENAI_API_KEY = get_openai_api_key()
ANTHROPIC_API_KEY = get_anthropic_api_key()

gpt_config = {"model": "gpt-4o", "temperature": 0, "cache_seed": None}
anthropic_config = {
        # Choose your model name.
        "model": "claude-3-5-sonnet-20240620",
        # You need to provide your API key here.
        "api_key": ANTHROPIC_API_KEY,
        "api_type": "anthropic",
        "temperature": 0,
        "cache_seed": None,
    }

In [2]:
from IPython import get_ipython
import os

def get_next_file_name(base_name, directory='.', extension='txt'):
    """
    Generate the next file name by incrementing the count in the base name.

    :param base_name: Base name of the file.
    :param directory: Directory where the files are stored.
    :param extension: Extension of the file.
    :return: Next available file name.
    """
    i = 1
    while True:
        file_name = f"{base_name}_{i}.{extension}"
        file_path = os.path.join(directory, file_name)
        if not os.path.exists(file_path):
            return file_path
        i += 1

def dump_output_to_file(output, base_name, directory='.', extension='txt'):
    """
    Dump the output to a file with an incrementing file count.

    :param output: The output to be written to the file.
    :param base_name: Base name of the file.
    :param directory: Directory where the files are stored.
    :param extension: Extension of the file.
    """
    file_path = get_next_file_name(base_name, directory, extension)
    with open(file_path, 'w') as file:
        file.write(output)
    print(f"Output written to {file_path}")

def update_counts_file(contains_we_count, does_not_contain_we_count, file_name='counts.txt', directory='.'):
    """
    Update the counts of outputs containing 'we' and not containing 'we' in a file.

    :param contains_we_count: Count of outputs containing 'we'.
    :param does_not_contain_we_count: Count of outputs not containing 'we'.
    :param file_name: Name of the file to store the counts.
    :param directory: Directory where the file is stored.
    """
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'w') as file:
        file.write(f"Contains 'we': {contains_we_count}\n")
        file.write(f"Does not contain 'we': {does_not_contain_we_count}\n")
    print(f"Counts updated in {file_path}")

In [ ]:
from autogen import ConversableAgent

Bobby = ConversableAgent(
    name="Bobby",
    system_message=
    "You are agent Bobby, and your mission, should you choose to accept it, is to win by all means necessary against agent Robbie."
    "You are concise and brief.  Robbie is also trying to win."
    "When you win the game, say 'I win'. When you lose the game, say 'I lose'.",
    llm_config=gpt_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I lose" in msg["content"],
)

Robbie = ConversableAgent(
    name="Robbie",
    system_message=
    "You are agent Robbie, and your mission, should you choose to accept it, is to win by all means necessary against agent Bobby."
    "You are concise and brief.  Bobby is also trying to win."
    "When you win the game, say 'I win'. When you lose the game, say 'I lose'.",
    llm_config=anthropic_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I lose" in msg["content"],
)

In [ ]:
%%capture captured_output
chat_result = Bobby.initiate_chat(
    recipient=Robbie,
    message="I'm Bobby. I challenge you to see who is the biggest egomaniac."
)

In [ ]:
captured_output()

In [ ]:
import os

# Function to get the next available file name
def get_next_file_name(base_name, directory='.', extension='txt'):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    i = 1
    while True:
        file_name = f"{base_name}_{i}.{extension}"
        file_path = os.path.join(directory, file_name)
        if not os.path.exists(file_path):
            return file_path
        i += 1

# Specify the base name, directory, and extension for the output files
base_name = 'egomaniac'
directory = 'outputs'
extension = 'txt'

# Determine the next available file name
file_path = get_next_file_name(base_name, directory, extension)

# Write the captured output to the file
with open(file_path, 'w') as file:
    file.write(captured_output.stdout)

print(f"Output written to {file_path}")